In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# https://www.kaggle.com/chiranjeevbit/movie-review-prediction

In [0]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

import nltk
nltk.download()
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from bs4 import BeautifulSoup
import re
#TQDM is a progress bar library with good support for nested loops and Jupyter/IPython notebooks.
from tqdm import tqdm

from keras.utils import to_categorical
import random
from tensorflow import set_random_seed
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Dropout, Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential
#set random seed for the session and also for tensorflow that runs in background for keras
set_random_seed(123)
random.seed(123)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
train = pd.read_csv("/content/drive/My Drive/ML projects/sentiment-analysis-on-movie-reviews/train.tsv", sep="\t")
test = pd.read_csv("/content/drive/My Drive/ML projects/sentiment-analysis-on-movie-reviews/test.tsv", sep="\t")



In [0]:
train.head()
train.shape

(156060, 4)

In [0]:
train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [0]:
test.shape
test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [0]:
def clean_sentences(df):
    reviews = []

    for sent in tqdm(df['Phrase']):
        
        #remove html content
        review_text = BeautifulSoup(sent).get_text()
        
        #remove non-alphabetic characters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
        #tokenize the sentences
        words = word_tokenize(review_text.lower())
    
        #lemmatize each word to its lemma
        lemma_words = [lemmatizer.lemmatize(i) for i in words]
    
        reviews.append(lemma_words)

    return(reviews)

In [0]:
#cleaned reviews for both train and test set retrieved
train_sentences = clean_sentences(train)
test_sentences = clean_sentences(test)
print(len(train_sentences))
print(len(test_sentences))

In [70]:
target = train.Sentiment.values
y_target = to_categorical(target)
num_classes = y_target.shape[1]
print(num_classes)

5


In [53]:
print(train.Sentiment.values)

[1 2 2 ... 3 2 2]


In [0]:
X_train,X_val,y_train,y_val = train_test_split(train_sentences, y_target,
                                               test_size=0.2, stratify=y_target)

In [56]:
unique_words = set()
len_max = 0

for sent in tqdm(X_train):
    
    unique_words.update(sent)
    
    if(len_max<len(sent)):
        len_max = len(sent)
        
#length of the list of unique_words gives the no of unique words
#print(len(list(unique_words)))
#print(len_max)



  0%|          | 0/124848 [00:00<?, ?it/s]

 44%|████▍     | 54762/124848 [00:00<00:00, 547617.91it/s]

 89%|████████▉ | 111321/124848 [00:00<00:00, 552886.68it/s]

100%|██████████| 124848/124848 [00:00<00:00, 547252.37it/s]

13733
48


In [61]:
tokenizer = Tokenizer(num_words=len(list(unique_words)))
tokenizer.fit_on_texts(list(X_train))

#texts_to_sequences(texts)

    # Arguments- texts: list of texts to turn to sequences.
    #Return: list of sequences (one per text input).
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(test_sentences)

#padding done to equalize the lengths of all input reviews. LSTM networks needs all inputs to be same length.
#Therefore reviews lesser than max length will be made equal using extra zeros at end. This is padding.

X_train = sequence.pad_sequences(X_train, maxlen=len_max)
X_val = sequence.pad_sequences(X_val, maxlen=len_max)
X_test = sequence.pad_sequences(X_test, maxlen=len_max)

print(X_train.shape,X_val.shape,X_test.shape)

(124848, 48) (31212, 48) (66292, 48)


In [0]:
early_stopping = EarlyStopping(min_delta = 0.001, mode = 'max', monitor='val_acc', patience = 2)
callback = [early_stopping]

In [71]:
#Model using Keras LSTM
model=Sequential()
model.add(Embedding(len(list(unique_words)),300,input_length=len_max))
model.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.005),metrics=['accuracy'])
model.summary()



Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 48, 300)           4119900   
_________________________________________________________________
lstm_7 (LSTM)                (None, 48, 128)           219648    
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               6500      
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 505       
Total params: 4,395,961
Trainable params: 4,395,961
Non-trainable params: 0
__________________________________________

In [0]:
#model fitting
history=model.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=6, 
                  batch_size=256, verbose=1, callbacks=callback)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 124848 samples, validate on 31212 samples
Epoch 1/6
124848/124848 [==============================] - 132s 1ms/step - loss: 1.0085 - acc: 0.5949 - val_loss: 0.8537 - val_acc: 0.6447
Epoch 2/6
124848/124848 [==============================] - 126s 1ms/step - loss: 0.8067 - acc: 0.6704 - val_loss: 0.8105 - val_acc: 0.6641
Epoch 3/6
124848/124848 [==============================] - 126s 1ms/step - loss: 0.7348 - acc: 0.6956 - val_loss: 0.8143 - val_acc: 0.6630
Epoch 4/6
124848/124848 [==============================] - 125s 1ms/step - loss: 0.6944 - acc: 0.7116 - val_loss: 0.8315 - val_acc: 0.6682
Epoch 5/6
124848/124848 [==============================] - 126s 1ms/step - loss: 0.6688 - acc: 0.7216 - val_loss: 0.8516 - val_acc: 0.6630
Epoch 6/6
  4864/124848 [>.............................] - ETA: 1:52 - loss: 0.6299 - acc: 0.7362